# Compute efficiency distributions for a proton sample

In [ ]:
#!python3 -m pip install --user uproot4 awkward1 mplhep

In [ ]:
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt

### Proton reconstruction flavors

There are two ways to reconstruct protons in PPS: _single-RP_ and _multi-RP_
The flag below controls which proton reconstruction is used.

In the single-RP reconstruction, the proton kinematics is only partially reconstructed from a single detector station. 
In the multi-RP reconstruction, both "near" and "far" tracking detector stations in one spectrometer arm are used. 

The second is more accurate, at the cost of acceptance. In this exercise we will see how to retrieve efficiency corrections in for multiRP protons. 

We will work on a MC file with quite large statistics and apply the corrections needed to compare it with data taken in the **2017B** period

In [ ]:
data_folder = "/eos/user/c/cmsdas/short-exercises/pps-protons-tutorial/data/"
fileName_signal = data_folder+"output-MC2017-Elastic-Non3+3-PreSel.h5"

#fileName_data = data_folder+"data/output-UL2017B-PreSel.h5"
#fileName_data = data_folder+"data/output-UL2017C1-PreSel.h5"
#fileName_data = data_folder+"data/output-UL2017F1-PreSel.h5"

#proton_selection = "SingleRP"
proton_selection = "MultiRP"

## Access the MC

In [ ]:
df_signal = None
event_counts_signal = None
selections_str_signal = None

with h5py.File( fileName_signal, 'r' ) as f:
    print ( list(f.keys()) )
    dset = f['protons']
    print ( dset.shape )
    print ( dset[:,:] )
    dset_columns = f['columns']
    print ( dset_columns.shape )
    columns = list( dset_columns )
    print ( columns )
    columns_str = [ item.decode("utf-8") for item in columns ]
    print ( columns_str )

    df_signal = pd.DataFrame( dset, columns=columns_str )
    
    df_signal = df_signal[ ['Run', 'LumiSection', 'EventNum', 'CrossingAngle', 
                            'MultiRP', 'Arm', 'RPId1', 'RPId2', 'TrackX1', 'TrackY1', 'TrackX2', 'TrackY2',
                            'Xi', 'T', 'ThX', 'ThY',
                            'Muon0Pt', 'Muon1Pt', 'InvMass', 'ExtraPfCands', 'Acopl', 'XiMuMuPlus', 'XiMuMuMinus'] ].astype( { "Run": "int64", "LumiSection": "int64", "EventNum": "int64", "MultiRP": "int32", "Arm": "int32", "RPId1": "int32", "RPId2": "int32", "ExtraPfCands": "int32" } )
    

### Proton efficiency extraction

We will evaluate the efficiency correction for _multi-RP_ protons. 

This has to be computed as the product of the efficiency of the "210 Far" detector station (silicon strips, in 2017) and the "multiRP efficiency".

The efficiency components for the strips is divided in two parts: multitrack and sensor. 
The former accounts the inefficiency caused by strips detectors not being able to reconstruct multiple tracks in the same event. This is a single numeric factor.
The sensor efficiency takes into account local inefficiencies caused, e.g., by radiation damage and is supplied as a 2D, xy map. 

The multiRP efficiency accounts instead for both the sensor efficiency of the 220 Far RP (pixels in 2017 and 2018) and the inefficiency component given by the proton chance of interacting while travelling from the near to the far RP. Again, this is supplied as a 2D, xy map.

A different measurement is available for each data period here included: 2017B, 2017C1, 2017F1.

Despite being counter-intuitive, it should be noted that, for 210 Far RP efficiencies one should use the xy coordinates measured in the 220 Far RP (TrackX2/TrackY2) and for multiRP the ones measured in the 210 Far RP (TrackX1/TrackY1). This is a consequence of the Tag&Probe method used to produce the efficiency corrections.

In [ ]:
import ROOT

# Efficiency correction files
strips_efficiency_file = ROOT.TFile.Open("/eos/home-c/cmsdas/short-exercises/pps-protons-tutorial/PreliminaryEfficiencies_July132020_1D2DMultiTrack.root")
multiRP_efficiency_file = ROOT.TFile.Open("/eos/home-c/cmsdas/short-exercises/pps-protons-tutorial/pixelEfficiencies_multiRP.root")

data_taking_period = "2017B"
year = data_taking_period[:4]

strips_multitrack_efficiency = {}
strips_sensor_efficiency = {}
multiRP_efficiency = {}

# Retrieve histograms from files and save them in dictionaries for future usage
for sector in ["45","56"]:
    rp_number = "3" if sector == "45" else "103"
    strips_multitrack_efficiency[sector] = strips_efficiency_file.Get("Strips/"+year+"/"+data_taking_period+"/h"+sector+"multitrackeff_"+data_taking_period+"_avg_RP"+rp_number).GetBinContent(1)
    strips_sensor_efficiency[sector] = strips_efficiency_file.Get("Strips/"+year+"/"+data_taking_period+"/h"+sector+"_"+data_taking_period+"_all_2D")
    multiRP_efficiency[sector] = multiRP_efficiency_file.Get("Pixel/"+year+"/"+data_taking_period+"/h"+sector+"_220_"+data_taking_period+"_all_2D")

In [ ]:
# Select only multiRP protons from dataframe
df_multiRP = df_signal[df_signal["MultiRP"] == 1]

# Create column for the efficiency
proton_strips_multitrack_efficiency = []
proton_strips_sensor_efficiency = []
proton_multiRP_efficiency = []
for row_n,row in df_multiRP.iterrows():
    sector = "45" if row["Arm"] == 0 else "56"
    proton_strips_multitrack_efficiency.append(strips_multitrack_efficiency[sector])
    proton_strips_sensor_efficiency.append(strips_sensor_efficiency[sector].GetBinContent(strips_sensor_efficiency[sector].FindBin(row["TrackX2"],row["TrackY2"])))
    proton_multiRP_efficiency.append(multiRP_efficiency[sector].GetBinContent(multiRP_efficiency[sector].FindBin(row["TrackX1"],row["TrackY1"])))

In [ ]:
# Add efficiency column to the new multiRP protons dataframe
df_multiRP_withEfficiency = df_multiRP.copy()
df_multiRP_withEfficiency["Strips_multitrack_efficiency"] = proton_strips_multitrack_efficiency 
df_multiRP_withEfficiency["Strips_sensor_efficiency"] = proton_strips_sensor_efficiency
df_multiRP_withEfficiency["MultiRP_efficiency"] = proton_multiRP_efficiency
df_multiRP_withEfficiency["Efficiency"] = df_multiRP_withEfficiency["Strips_multitrack_efficiency"] * df_multiRP_withEfficiency["Strips_sensor_efficiency"] * df_multiRP_withEfficiency["MultiRP_efficiency"]

In [ ]:
# Show a few lines of the new dataframe
df_multiRP_withEfficiency[:20]

### Plot the efficiency vs. Xi distributions

In [ ]:
fig, axis = plt.subplots(figsize=(10,10))
n_bins = 10
arm = 0

# Select only protons with Arm == 0
df_multiRP_withEfficiency_arm = df_multiRP_withEfficiency[df_multiRP_withEfficiency["Arm"] == arm]

# Bin data depending on xi -> xi_counts = bin content
xi_counts, xi_bins, *_ = axis.hist( df_multiRP_withEfficiency_arm["Xi"], bins=n_bins,visible=False)

# Get bin contents while weighting on the strips multitrack efficiency of each entry
strips_multitrack_efficiency_counts, *_ = axis.hist( df_multiRP_withEfficiency_arm["Xi"], bins=n_bins,weights=df_multiRP_withEfficiency_arm["Strips_multitrack_efficiency"],visible=False)
# Get bin contents while weighting on the strips sensor efficiency of each entry
strips_sensor_efficiency_counts, *_ = axis.hist( df_multiRP_withEfficiency_arm["Xi"], bins=n_bins,weights=df_multiRP_withEfficiency_arm["Strips_sensor_efficiency"],visible=False)
# Get bin contents while weighting on the multiRP efficiency of each entry
multiRP_efficiency_counts, *_ = axis.hist( df_multiRP_withEfficiency_arm["Xi"], bins=n_bins,weights=df_multiRP_withEfficiency_arm["MultiRP_efficiency"],visible=False)

# Compute bin centers and averages
xi_bin_centers = [(xi_bins[i] + xi_bins[i+1])/2 for i in range(len(xi_bins)-1)]
strips_multitrack_avg_efficiency_counts = strips_multitrack_efficiency_counts / xi_counts
strips_sensor_avg_efficiency_counts = strips_sensor_efficiency_counts / xi_counts
multiRP_avg_efficiency_counts = multiRP_efficiency_counts / xi_counts
total_efficiency = strips_multitrack_avg_efficiency_counts * strips_sensor_avg_efficiency_counts * multiRP_avg_efficiency_counts

# Plot efficiencies
axis.plot(xi_bin_centers,strips_multitrack_avg_efficiency_counts, label="Strips Multitrack Efficiency")
axis.plot(xi_bin_centers,strips_sensor_avg_efficiency_counts, label="Strips Sensor Efficiency")
axis.plot(xi_bin_centers,multiRP_avg_efficiency_counts, label="MultiRP Efficiency")
axis.plot(xi_bin_centers,total_efficiency, label="Total Efficiency")


# Add labels and legend
ylim = axis.set_ylim([0,1])
axis.set_ylabel( "Efficiency", fontsize=20 )
axis.set_xlabel( r"$\xi$", fontsize=20 )
l = plt.legend(fontsize=20)